<a href="https://colab.research.google.com/github/ttury/Deep-Learning-For-Natural-Language-Processing/blob/master/Text_Generation_using_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
from tensorflow.keras.utils import to_categorical

text="""경마장에 있는 말이 뛰고 있다
그의 말이 법이다
가는 말이 고와야 오는 말이 곱다"""

tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])
print(tokenizer.word_index)

vocab_size = len(tokenizer.word_index) + 1
# 케라스 토크나이저의 정수 인코딩은 인덱스가 1부터 시작하지만,
# 케라스 원-핫 인코딩은 인덱스가 0부터 시작하기 때문에
# 인덱스를 1부터 시작하기 위해 단어 집합의 크기보다 1 크게 지정

print('단어 집합의 크기 : %d' % vocab_size)

In [ ]:
sequences = list()
for line in text.split('\n'):
  print(line)
  encoded = tokenizer.texts_to_sequences([line])[0]
  print(encoded)
  # 정수 인코딩, 2차원 리스트지만 한 문장밖에 없으므로 첫 원소가 전체 데이터와 같음 
  for i in range(1, len(encoded)):
    sequence = encoded[:i+1] # 한 문장의 완성 과정을 분리 ex) 그의 / 그의 말이 / 그의 말이 법이다
    sequences.append(sequence)

print()
print('학습에 사용할 샘플의 개수: {}'.format(len(sequences)))
print(sequences)

In [ ]:
max_len = max(len(sequence) for sequence in sequences) # 가장 긴 샘플을 기준으로 샘플 길이를 패딩 -> 6으로 패딩
sequences = pad_sequences(sequences, maxlen=max_len, padding='pre') # pre 값을 주면 앞에서부터 0으로 채움

print(sequences)

In [ ]:
# 각 샘플의 마지막 단어를 레이블로 분리해서 정답으로 사용

sequences = np.array(sequences)
x = sequences[:, :-1]
y = sequences[:, -1]

print(x)
print(y)

In [ ]:
# one-hot-encoding

y = to_categorical(y, num_classes=vocab_size)
print(y)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, SimpleRNN

model = Sequential()
model.add(Embedding(vocab_size, 10, input_length=max_len-1)) # (vocab_size x 10) 크기의 임베딩 행렬
# x와 y를 분리해서 x의 길이가 1 줄었다
model.add(SimpleRNN(32)) # hidden_size = 32
model.add(Dense(vocab_size, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(x, y, epochs=200, verbose=2)

In [41]:
def generate_sent(model, tokenizer, current_word, n):
  sentence = current_word

  for _ in range(n):
    encoded = tokenizer.texts_to_sequences([current_word])[0]
    encoded = pad_sequences([encoded], maxlen=5, padding='pre')
    result = model.predict_classes(encoded, verbose=0)
    for word, index in tokenizer.word_index.items():
      if index == result:
        current_word = current_word + ' ' + word
        sentence = sentence + ' ' + word
        break

  return sentence

In [1]:
print(generate_sent(model, tokenizer, '경마장에', 4))
print(generate_sent(model, tokenizer, '그의', 2))
print(generate_sent(model, tokenizer, '가는', 5))

NameError: ignored